# tf.Keras

Doc [here](https://www.tensorflow.org/guide/keras)

In [80]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import json
import pprint

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


### A simple model

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, 
                       activation='relu',
                       kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model.add(layers.Dense(64, 
                       activation='sigmoid',
                       bias_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(layers.Dense(32,
                       kernel_initializer='orthogonal',
                       bias_initializer=tf.keras.initializers.constant(2.0)))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


### Simple training

In [6]:
data = np.random.random((1000,32))
labels = np.random.random((1000,1))

model.fit(data,
          labels,
          epochs=3,
          batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
1000/1000 [==============================] - 1s 699us/sample - loss: 18.0976 - acc: 0.0920
Epoch 2/3
1000/1000 [==============================] - 0s 112us/sample - loss: 13.5388 - acc: 0.0330
Epoch 3/3
1000/1000 [==============================] - 0s 113us/sample - loss: 13.0527 - acc: 0.1090


### Training with validation

In [8]:
data = np.random.random((1000,32))
labels = np.random.random((1000,10))
val_data = np.random.random((100,32))
val_labels = np.random.random((100,10))

model.fit(data,
          labels,
          epochs=3,
          batch_size=32,
          validation_data=(val_data,
                           val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/3
1000/1000 [==============================] - 0s 262us/sample - loss: 12.7066 - acc: 0.1000 - val_loss: 12.7606 - val_acc: 0.0400
Epoch 2/3
1000/1000 [==============================] - 0s 138us/sample - loss: 12.3861 - acc: 0.0850 - val_loss: 12.4604 - val_acc: 0.0800
Epoch 3/3
1000/1000 [==============================] - 0s 110us/sample - loss: 12.1160 - acc: 0.0840 - val_loss: 12.2219 - val_acc: 0.0500


### Variations on that

For instance, it is possible to define the model in one go, as an array of layers.

In [13]:
model2 = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(64, activation='sigmoid'),
    layers.Dense(10, activation='softmax')
])

model2.compile(optimizer=tf.keras.optimizers.Adagrad(), # why not this one
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# reusing the same dummy data
model2.fit(data,labels,epochs=3,batch_size=53,
           validation_data=(val_data,val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/3
1000/1000 [==============================] - 1s 568us/sample - loss: 11.5368 - acc: 0.0920 - val_loss: 11.7015 - val_acc: 0.0800
Epoch 2/3
1000/1000 [==============================] - 0s 148us/sample - loss: 11.4609 - acc: 0.1030 - val_loss: 11.6988 - val_acc: 0.1100
Epoch 3/3
1000/1000 [==============================] - 0s 109us/sample - loss: 11.4558 - acc: 0.1140 - val_loss: 11.6984 - val_acc: 0.0900


In [15]:
# I can just recompile it with another optimizer
model2.compile(optimizer=tf.train.AdamOptimizer(0.001), # why not this one
               loss='mse',
               metrics=['mae'])

# reusing the same dummy data
model2.fit(data,labels,epochs=3,batch_size=53,
           validation_data=(val_data,val_labels))

Instructions for updating:
Use tf.cast instead.
Train on 1000 samples, validate on 100 samples
Epoch 1/3
1000/1000 [==============================] - 1s 671us/sample - loss: 0.2403 - mean_absolute_error: 0.4071 - val_loss: 0.2517 - val_mean_absolute_error: 0.4171
Epoch 2/3
1000/1000 [==============================] - 0s 66us/sample - loss: 0.2402 - mean_absolute_error: 0.4071 - val_loss: 0.2516 - val_mean_absolute_error: 0.4172
Epoch 3/3
1000/1000 [==============================] - 0s 70us/sample - loss: 0.2401 - mean_absolute_error: 0.4070 - val_loss: 0.2515 - val_mean_absolute_error: 0.4170


In [17]:
# and yet another
model2.compile(optimizer=tf.train.RMSPropOptimizer(0.01), # why not this one
               loss=tf.keras.losses.categorical_crossentropy,
               metrics=[tf.keras.metrics.categorical_accuracy])

# reusing the same dummy data
model2.fit(data,labels,epochs=3,batch_size=12,
           validation_data=(val_data,val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/3
1000/1000 [==============================] - 1s 877us/sample - loss: 11.4908 - categorical_accuracy: 0.1310 - val_loss: 12.0605 - val_categorical_accuracy: 0.1200
Epoch 2/3
1000/1000 [==============================] - 0s 250us/sample - loss: 11.5362 - categorical_accuracy: 0.1010 - val_loss: 11.7051 - val_categorical_accuracy: 0.0800
Epoch 3/3
1000/1000 [==============================] - 0s 480us/sample - loss: 11.4570 - categorical_accuracy: 0.1020 - val_loss: 11.6884 - val_categorical_accuracy: 0.1100


### Import tf.data.Dataset

In [19]:
keras_ds = tf.data.Dataset.from_tensor_slices((data, labels))
keras_ds = keras_ds.batch(32)
keras_ds = keras_ds.repeat()

model.fit(keras_ds,
          epochs=3,
          steps_per_epoch=50) # why do we need to specify this?

Epoch 1/3
50/50 [==============================] - 1s 11ms/step - loss: 11.6093 - acc: 0.0984
Epoch 2/3
50/50 [==============================] - 0s 3ms/step - loss: 11.5074 - acc: 0.0921
Epoch 3/3
50/50 [==============================] - 0s 3ms/step - loss: 11.5151 - acc: 0.0914


In the doc:

> Here, the fit method uses the steps_per_epoch argument—this is the number of training steps the model runs before it moves to the next epoch. Since the Dataset yields batches of data, this snippet does not require a batch_size.

With validation now:

In [24]:
keras_ds = tf.data.Dataset.from_tensor_slices((data,labels))
keras_ds = keras_ds.batch(32).repeat()
val_ds = tf.data.Dataset.from_tensor_slices((val_data,val_labels))
val_ds = val_ds.batch(32).repeat()

model.fit(keras_ds,
          epochs=3,
          steps_per_epoch=100,
          validation_data=val_ds,
          validation_steps=3)

Epoch 1/3
100/100 [==============================] - 1s 11ms/step - loss: 11.4831 - acc: 0.0975 - val_loss: 11.7040 - val_acc: 0.0833
Epoch 2/3
100/100 [==============================] - 0s 3ms/step - loss: 11.4932 - acc: 0.1049 - val_loss: 11.5457 - val_acc: 0.0735
Epoch 3/3
100/100 [==============================] - 0s 3ms/step - loss: 11.4786 - acc: 0.1065 - val_loss: 11.6119 - val_acc: 0.0588


### Evaluate and predict

In [26]:
eval_d = np.random.random((1000,32))
eval_l = np.random.random((1000,10))

model.evaluate(eval_d, 
               eval_l, 
               batch_size=32)
model.evaluate(keras_ds, steps=32)

32/32 [==============================] - 0s 8ms/step - loss: 11.4741 - acc: 0.1180


[11.474053680896759, 0.118]

In [27]:
result = model.predict(eval_d, batch_size=64)
print(result.shape)

(1000, 10)


### Advanced models

In [28]:
inputs = tf.keras.Input(shape=(32,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(eval_d, eval_l, batch_size=32, epochs=3)

Epoch 1/3
1000/1000 [==============================] - 1s 586us/sample - loss: 11.5404 - acc: 0.0930
Epoch 2/3
1000/1000 [==============================] - 0s 92us/sample - loss: 11.4977 - acc: 0.0830
Epoch 3/3
1000/1000 [==============================] - 0s 87us/sample - loss: 11.4762 - acc: 0.0930


### Subclassing a model

As usual, more modularity and fine-grained control comes at the cost of more potential mistakes.

In [52]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.dense_1 = layers.Dense(32, activation='relu')
        self.dense_2 = layers.Dense(num_classes, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self, input_shape):
        # needed to use subc model in a functional-style model
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

In [55]:
sub = MyModel(num_classes=10)

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(eval_d, eval_l, batch_size=32, epochs=3)

Epoch 1/3
1000/1000 [==============================] - 0s 452us/sample - loss: 11.4409 - acc: 0.1140
Epoch 2/3
1000/1000 [==============================] - 0s 87us/sample - loss: 11.4406 - acc: 0.1200
Epoch 3/3
1000/1000 [==============================] - 0s 87us/sample - loss: 11.4413 - acc: 0.1240


### Custom layers

In [62]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel',
                                      shape=shape,
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    # necessary for serialization
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [63]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')
])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
               loss='categorical_crossentropy',
               metrics=['accuracy'])
model.fit(eval_d, eval_l, batch_size=32, epochs=3)

Epoch 1/3
1000/1000 [==============================] - 0s 406us/sample - loss: 11.4835 - acc: 0.1000
Epoch 2/3
1000/1000 [==============================] - 0s 85us/sample - loss: 11.4640 - acc: 0.0990
Epoch 3/3
1000/1000 [==============================] - 0s 75us/sample - loss: 11.4579 - acc: 0.1120


### Callbacks

In [70]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

model.fit(keras_ds,epochs=5, # since from dataset, no batch
          steps_per_epoch=50, # must be specified
          callbacks=callbacks,
          validation_data=val_ds,
          validation_steps=3) # same as steps_per_epoch

Epoch 1/5
50/50 [==============================] - 1s 11ms/step - loss: 11.4871 - acc: 0.1294 - val_loss: 11.7084 - val_acc: 0.0521
Epoch 2/5
50/50 [==============================] - 0s 4ms/step - loss: 11.4410 - acc: 0.1250 - val_loss: 11.5548 - val_acc: 0.0735
Epoch 3/5
50/50 [==============================] - 0s 5ms/step - loss: 11.4758 - acc: 0.1377 - val_loss: 11.6230 - val_acc: 0.0588
Epoch 4/5
50/50 [==============================] - 0s 3ms/step - loss: 11.4753 - acc: 0.1482 - val_loss: 11.7142 - val_acc: 0.1029


In [77]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(eval_d, eval_l, epochs=5, batch_size=32)

model.save_weights('./dummy_keras/model')
# # also possible as h5
# model.save_weights('./dummy_keras/model.h5', save_format='h5')

Epoch 1/5
1000/1000 [==============================] - 1s 503us/sample - loss: 11.5334 - acc: 0.1060
Epoch 2/5
1000/1000 [==============================] - 0s 99us/sample - loss: 11.4909 - acc: 0.1060
Epoch 3/5
1000/1000 [==============================] - 0s 101us/sample - loss: 11.4783 - acc: 0.1090
Epoch 4/5
1000/1000 [==============================] - 0s 100us/sample - loss: 11.4696 - acc: 0.1160
Epoch 5/5
1000/1000 [==============================] - 0s 95us/sample - loss: 11.4642 - acc: 0.1090


In [78]:
model.load_weights('./dummy_keras/model')
# model.load_weights('./dummy_keras/model.h5')
model.fit(eval_d, eval_l, epochs=3, batch_size=32)

Epoch 1/3
1000/1000 [==============================] - 0s 102us/sample - loss: 11.4588 - acc: 0.1110
Epoch 2/3
1000/1000 [==============================] - 0s 99us/sample - loss: 11.4554 - acc: 0.1290
Epoch 3/3
1000/1000 [==============================] - 0s 101us/sample - loss: 11.4512 - acc: 0.1300


In [81]:
json_string = model.to_json()
print(json_string)
print()
pprint.pprint(json.loads(json_string))

{"class_name": "Sequential", "config": {"name": "sequential_15", "layers": [{"class_name": "Dense", "config": {"name": "dense_43", "trainable": true, "batch_input_shape": [null, 32], "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_44", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constrai

In [84]:
# possible then to create a new model from it
new_model = tf.keras.models.model_from_json(json_string)

new_model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
new_model.fit(eval_d, eval_l, batch_size=32, epochs=3)

Epoch 1/3
1000/1000 [==============================] - 1s 764us/sample - loss: 11.5626 - acc: 0.1050
Epoch 2/3
1000/1000 [==============================] - 0s 117us/sample - loss: 11.4918 - acc: 0.1230
Epoch 3/3
1000/1000 [==============================] - 0s 104us/sample - loss: 11.4775 - acc: 0.1210


In [85]:
# same with yaml

yaml_string = model.to_yaml()
print(yaml_string)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple [null, 32]
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {dtype: float32, seed: null}
      kernel_regularizer: null
      name: dense_43
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {dtype: float32, seed: null}
  

In [86]:
# possible then to create a new model from it
new_model = tf.keras.models.model_from_yaml(yaml_string)

new_model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
new_model.fit(eval_d, eval_l, batch_size=32, epochs=3)

Epoch 1/3
1000/1000 [==============================] - 1s 537us/sample - loss: 11.5587 - acc: 0.0930
Epoch 2/3
1000/1000 [==============================] - 0s 215us/sample - loss: 11.5118 - acc: 0.0980
Epoch 3/3
1000/1000 [==============================] - 0s 176us/sample - loss: 11.4910 - acc: 0.1100


In [89]:
entire = tf.keras.Sequential([
    layers.Dense(64, activation='tanh', input_shape=(32,)),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(eval_d, eval_l, batch_size=32, epochs=3)
print('done training!')
print('-'*30)

model.save('./dummy_keras/entire.h5')
print('reloading model, training again:')
model_loaded = tf.keras.models.load_model('./dummy_keras/entire.h5')
model_loaded.fit(eval_d, eval_l, batch_size=32, epochs=3)

Epoch 1/3
1000/1000 [==============================] - 1s 634us/sample - loss: 11.4352 - acc: 0.1420
Epoch 2/3
1000/1000 [==============================] - 0s 181us/sample - loss: 11.4312 - acc: 0.1570
Epoch 3/3
1000/1000 [==============================] - 0s 190us/sample - loss: 11.4289 - acc: 0.1570
done training!
------------------------------
reloading model, training again:
Epoch 1/3
1000/1000 [==============================] - 1s 600us/sample - loss: 11.4286 - acc: 0.1570
Epoch 2/3
1000/1000 [==============================] - 0s 96us/sample - loss: 11.4246 - acc: 0.1640
Epoch 3/3
1000/1000 [==============================] - 0s 102us/sample - loss: 11.4218 - acc: 0.1520
